<a href="https://colab.research.google.com/github/contractorwolf/AngularAPIViewer/blob/master/Colab_Voice_Cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


 
#### I have made edits to the orginal codebase, located here:
https://github.com/CorentinJ/Real-Time-Voice-Cloning


#### My edits were mainly related to getting the code running fully on Google Colab.  My focus was using features made for Colab to record and play audio files.  This way you can demo the end-to-end training of a voice model using the browser to record the voice samples and play the result. 


#### Make sure your GPU is enabled as the runtime in Colab 
#### Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU


install required audio libraries

In [1]:
!apt-get install libportaudio2
print("INSTALL COMPLETE")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (137 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1) ...
Setting up libportaudio2:amd64 (19.6.0-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
INSTALL COMPLETE


Download the repo with my changes to the python file used to input the wav files. I added more command line parameters so that you can easily use colab to record and serve the files to the ML model. If you would like to see the original project: https://github.com/CorentinJ/Real-Time-Voice-Cloning

My changes simply enhance the cli version demo to work differently with Colab.  

In [2]:
!git clone https://github.com/contractorwolf/Real-Time-Voice-Cloning.git  
  
print("DOWNLOAD COMPLETE")

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 2421 (delta 11), reused 13 (delta 6), pack-reused 2402
Receiving objects: 100% (2421/2421), 360.25 MiB | 37.94 MiB/s, done.
Resolving deltas: 100% (1338/1338), done.
DOWNLOAD COMPLETE


Drop into that folder to install requirements

In [3]:
cd Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning





Install the requirements for this repository.


In [4]:
!pip install -r requirements.txt

print("REQUIREMENTS INSTALLED")


     |████████████████████████████████| 377.0MB 65kB/s 
     |████████████████████████████████| 1.4MB 33.4MB/s 
     |████████████████████████████████| 71kB 26.6MB/s 
     |████████████████████████████████| 245kB 41.0MB/s 
     |████████████████████████████████| 62.1MB 635kB/s 
     |████████████████████████████████| 204kB 41.9MB/s 
     |████████████████████████████████| 71kB 28.4MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.8-cp36-none-any.whl size=1350603 sha256=56900577a63cb833cbd79739cdc6ef3cbf0c02ae18b779aae8db541b8e7e0fdd
  Stored in directory: /root/.cache/pip/wheels/ee/87/ce/a5023722374ca73b57fc8d4284ba6f973c01219b3c385a07e0
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71456 sha256=55a7c3e90d4b1e387dbfd21d964ba0505ebccc827ee18ad6c545f0ab162d5bfd
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl



Download and unzip the pretrained voice model that used audiobooks (text and voice) as the source. This source lacks tone, but is usable. You are going to fine tune this pretrained model to your voice recording to make the cloned voice.  You can then pass text to that voice model and it will read them in your voice. 

Download the generic pretrained model by running the code block below.

In [5]:
#!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc #< original model hosted here
!gdown https://drive.google.com/uc?id=1dmjCLeh4w5xWOc7daA2GQPF1IqXsYTxi #< my copy of pretrained model

print("UNZIPPING: pretrained.zip")

!unzip pretrained.zip

print("UNZIP COMPLETE")

Downloading...
From: https://drive.google.com/uc?id=1dmjCLeh4w5xWOc7daA2GQPF1IqXsYTxi
To: /content/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:01, 194MB/s]
UNZIPPING: pretrained.zip
Archive:  pretrained.zip
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved_models/pretrained/p

Load the libaries and fancy javascript used for attaching to the browser to do the audio recording.  You will then record the 3 samples in the following code blocks.


In [7]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython.display as ipd

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(filename, sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open(filename,'wb') as f:
    f.write(b)
  return 'FILE RECORDED: ' + filename


print("libraries and javascript loaded")

libraries and javascript loaded


Create an audio recording in the browser for the sample audio file (voice to be cloned) by playing the next code block. It will immediately start recording a file named "input.wav" to be used in future steps by the encoder. Read the text below until the block of code has completed (10 seconds).

READ THIS:

# The Pro is an upgraded version of the Classic. In the pro model you get a electrical system, a super powerful rear hub brushless motor with tons of torque, an easy-access battery with port for phone charging, and Fat Tires with puncture-resistant casings. 


In [8]:
record('input.wav', sec=10)
print("recording complete")
ipd.Audio('input.wav') 

<IPython.core.display.Javascript object>

recording complete


Create the second voice sample (you need 3). 

Read the text below until the block of code has completed (10 seconds).

READ THIS:

# The cafe racer movement is not entirely identical to the old British style. It blends different types of motorcycling together, including custom, flat track, classic and dirt bikes along with the original cafe racer concept. 

In [0]:
record('input2.wav', sec=10)
print("recording complete")
ipd.Audio('input2.wav') 

Create the third voice sample (final). 

Read the text below until the block of code has completed (10 seconds).

READ THIS:

# This uptown branch of the YMCA features amenities such as an adults-only lap pool, a variety of fitness classes, towel service and an in-house cafe with fresh and healthy lunch and breakfast options. 

In [0]:
record('input3.wav', sec=10)
print("recording complete")
ipd.Audio('input3.wav') 

Executing the python file below will create a voice signature from the 3 samples and use the input text to generate a wave file output.wav  

The output will be located here: 
/content/Real-Time-Voice-Cloning/output.wav

you will be able to play that output file in the browser once it has been generated.



In [17]:
#!python demo_cli.py --no_sound
#!python demo_colab.py --no_sound
#!python demo_colab_sample.py --no_sound

#!python convert.py \
#  --textin="there is nothing to fear, but fear itself" \
  #--no_sound 
  #--voicein="/content/Real-Time-Voice-Cloning/sample.wav" \
  #--out="output.wav"

!python translate.py \
  --textin="The Pro is an upgraded version of the Classic. In the pro model you get a electrical system"
  #--textin="i know you like to take the bus, but what does that have to do with me" \
  #--voicein="input.mp3" #the translate.py file has a default value of: "input.wav", only change it if needed

  #--textin="to be or not to be, that is the question" # shorter, too short, long gap in middle
  #--textin="Fourscore and seven years ago our fore fathers brought forth, on this continent, a new nation" #decent, if a bit fast
  #--textin="Fourscore and seven years ago our forefathers brought forth, on this continent" # BETTER LENGTH



import IPython.display as ipd
ipd.Audio('output.wav') # load a local WAV file


Arguments:
    enc_model_fpath:   encoder/saved_models/pretrained.pt
    syn_model_dir:     synthesizer/saved_models/logs-pretrained
    voc_model_fpath:   vocoder/saved_models/pretrained/pretrained.pt
    out:               output.wav
    textin:            The Pro is an upgraded version of the Classic. In the pro model you get a electrical system
    voicein:           input.wav


args.textin:  The Pro is an upgraded version of the Classic. In the pro model you get a electrical system

args.voicein:  input.wav

args.out:  output.wav
Running a test of your configuration...

Found 1 GPUs available. Using GPU 0 (Tesla K80) of compute capability 3.7 with 12.0Gb total memory.

Preparing the encoder, the synthesizer and the vocoder...
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt
args.voicein:  input.wav
in_fpa

Play output file which should be the incoming_text modeled with the voice signature file

In [16]:
import IPython.display as ipd
ipd.Audio('input.wav') # load a local WAV file

In [0]:
import IPython.display as ipd
ipd.Audio('output.wav') # load a local WAV file